In [269]:
import requests
import pandas as pd
import numpy as np

## Парсим файл и задаем формат

In [270]:
df = pd.read_json('..\\data\\auto.json', orient='records')
# df = df[df['Model'].notnull()]
df.reset_index(drop=True,inplace=True)
pd.options.display.float_format = '{:,.2f}'.format
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus


## Генерация данных

In [271]:
np.random.seed(21)
indices = [np.random.randint(len(df)) for i in range(200)]
# print(indices)
data = {'CarNumber': [df['CarNumber'][i] for i in indices]}
# print(data)
data.update({col: [np.random.choice(df[col]) for i in range(200)]
             for col in ['Refund', 'Fines']})
# print(data)
data.update({col: [df[col][i] for i in indices] for col in ['Make', 'Model']})
# print(data)
df2 = pd.DataFrame(data)
df2.tail()

,CarNumber,Refund,Fines,Make,Model
195,M942OT152RUS,1,500.00,Ford,Focus
196,Y187O8161RUS,2,600.00,Ford,Focus
197,7064C8197RUS,1,"39,600.00",Volkswagen,Passat
198,8437XX154RUS,2,"13,200.00",Ford,Focus
199,C410X938RUS,1,"8,594.59",Ford,Focus


In [272]:
concat_rows = pd.concat([df, df2])
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
dtype: int64

## Генерация лет

In [273]:
fines = concat_rows
fines['Year'] = pd.Series([np.random.randint(1980, 2020) for i in range(925)])
fines.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        914
Year         925
dtype: int64

## Фамилии

In [274]:
buf = fines.groupby(['CarNumber']).agg({'Refund' : ['count']})
# print(len(buf))
df = pd.read_json('..\\data\\surname.json', orient='columns')
df.columns = df.iloc[0] 
df = df[1:]
tt = fines.drop_duplicates(subset='CarNumber')
owners = pd.DataFrame(
    {'CarNumber': tt['CarNumber'][:len(buf)],
     'SURNAME': [np.random.choice(df['NAME']) for i in range(len(buf))]}
     )
print(owners)
print(len(buf))
len(owners)

        CarNumber  SURNAME
0    Y163O8161RUS    ADAMS
1     E432XX77RUS   NGUYEN
2     7184TT36RUS   ROGERS
3    X582HE161RUS  EDWARDS
4    92918M178RUS   BROOKS
..            ...      ...
715  O136HO197RUS    JAMES
719  O22097197RUS    PATEL
721  M0309X197RUS   NELSON
722  O673E8197RUS   WATSON
723  8610T8154RUS  EDWARDS

[531 rows x 2 columns]
531


531

In [275]:
data1 = pd.Series(['YQY187O8161RUS', 'YQY187O8162RUS', 'YQY187O8163RUS', 'YQY187O8164RUS', 'YQY187O8165RUS'], name='CarNumber')
data2 = pd.Series([np.random.choice(fines['Refund']) for i in range(5)], name= 'Refund')
data3 = pd.Series([np.random.choice(fines['Fines']) for i in range(5)], name= 'Fines')
data4 = pd.Series([np.random.choice(fines['Make']) for i in range(5)], name= 'Make')
data5 = pd.Series([np.random.choice(fines['Model']) for i in range(5)], name= 'Model')
data6 = pd.Series([np.random.choice(fines['Year']) for i in range(5)], name= 'Year')


df3 = pd.DataFrame([data1, data2, data3, data4, data5, data6])
fines = pd.concat([fines, df3.T])
fines.tail()

,CarNumber,Refund,Fines,Make,Model,Year
0,YQY187O8161RUS,2,"1,600.00",Skoda,Focus,2012
1,YQY187O8162RUS,2,"2,000.00",Ford,Focus,1987
2,YQY187O8163RUS,2,"8,600.00",Ford,Focus,1989
3,YQY187O8164RUS,2,"1,300.00",Ford,Focus,1999
4,YQY187O8165RUS,1,"5,500.00",Ford,Focus,2007


In [276]:
owners = owners.drop(owners.tail(23).index)
data1 = pd.Series(['PY33187O8161RUS', 'PY13387O8162RUS', 'PY13387O8163RUS'], name='CarNumber')
data2 = pd.Series([np.random.choice(owners['SURNAME']) for i in range(3)], name= 'SURNAME')
df3 = pd.DataFrame([data1, data2])
owners = pd.concat([owners, df3.T])
owners.tail()

,CarNumber,SURNAME
681,T914CT197RUS,TAYLOR
682,E41977152RUS,MENDOZA
0,PY33187O8161RUS,BENNETT
1,PY13387O8162RUS,RAMOS
2,PY13387O8163RUS,ROSS


In [277]:
len(fines)
# fines.set_index(keys='CarNumber', inplace= True)
# owners.set_index(keys='CarNumber', inplace= True)

930

## Сверка двух таблиц

In [278]:
new_df = pd.merge(left=fines, right=owners, how='inner', on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1998,ADAMS
1,Y163O8161RUS,2,"1,600.00",Ford,Focus,2003,ADAMS
2,Y163O8161RUS,2,"3,000.00",Ford,Focus,2012,ADAMS
3,Y163O8161RUS,2,"8,594.59",Ford,Focus,1999,ADAMS
4,Y163O8161RUS,2,"3,300.00",Ford,Focus,2006,ADAMS
...,...,...,...,...,...,...,...
895,9975CC77RUS,2,"7,000.00",Ford,Focus,2007,RUIZ
896,T195OK161RUS,1,"6,700.00",Ford,Mondeo,1983,JONES
897,T914CT197RUS,2,"7,200.00",Toyota,Camry,1986,TAYLOR
898,T914CT197RUS,1,"10,600.00",Toyota,Camry,2008,TAYLOR


In [279]:
new_df = pd.merge(left=fines, right=owners, how='outer', on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1998,ADAMS
1,Y163O8161RUS,2,"1,600.00",Ford,Focus,2003,ADAMS
2,Y163O8161RUS,2,"3,000.00",Ford,Focus,2012,ADAMS
3,Y163O8161RUS,2,"8,594.59",Ford,Focus,1999,ADAMS
4,Y163O8161RUS,2,"3,300.00",Ford,Focus,2006,ADAMS
...,...,...,...,...,...,...,...
928,YQY187O8164RUS,2,"1,300.00",Ford,Focus,1999,NaN
929,YQY187O8165RUS,1,"5,500.00",Ford,Focus,2007,NaN
930,PY33187O8161RUS,NaN,NaN,NaN,NaN,NaN,BENNETT
931,PY13387O8162RUS,NaN,NaN,NaN,NaN,NaN,RAMOS


In [280]:
new_df = pd.merge(left=fines, right=owners, how='left', on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1998,ADAMS
1,E432XX77RUS,1,"6,500.00",Toyota,Camry,1996,NGUYEN
2,7184TT36RUS,1,"2,100.00",Ford,Focus,2014,ROGERS
3,X582HE161RUS,2,"2,000.00",Ford,Focus,1996,EDWARDS
4,92918M178RUS,1,"5,700.00",Ford,Focus,1984,BROOKS
...,...,...,...,...,...,...,...
925,YQY187O8161RUS,2,"1,600.00",Skoda,Focus,2012,NaN
926,YQY187O8162RUS,2,"2,000.00",Ford,Focus,1987,NaN
927,YQY187O8163RUS,2,"8,600.00",Ford,Focus,1989,NaN
928,YQY187O8164RUS,2,"1,300.00",Ford,Focus,1999,NaN


In [281]:
new_df = pd.merge(left=fines, right=owners, how='right', on='CarNumber')
new_df

,CarNumber,Refund,Fines,Make,Model,Year,SURNAME
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1998,ADAMS
1,Y163O8161RUS,2,"1,600.00",Ford,Focus,2003,ADAMS
2,Y163O8161RUS,2,"3,000.00",Ford,Focus,2012,ADAMS
3,Y163O8161RUS,2,"8,594.59",Ford,Focus,1999,ADAMS
4,Y163O8161RUS,2,"3,300.00",Ford,Focus,2006,ADAMS
...,...,...,...,...,...,...,...
898,T914CT197RUS,1,"10,600.00",Toyota,Camry,2008,TAYLOR
899,E41977152RUS,2,"2,400.00",Ford,Focus,2011,MENDOZA
900,PY33187O8161RUS,NaN,NaN,NaN,NaN,NaN,BENNETT
901,PY13387O8162RUS,NaN,NaN,NaN,NaN,NaN,RAMOS


In [284]:
pd.pivot_table(fines, values='Fines', index=['Make', 'Model'], columns=['Year'],
               aggfunc=np.sum)

Year                    1980      1981       1982      1983      1984  \
Make       Model                                                        
Ford       Focus   40,694.59 37,000.00 216,989.17 43,300.00 98,194.59   
           Mondeo   1,100.00       NaN        NaN  6,700.00       NaN   
Skoda      Focus         NaN       NaN        NaN       NaN       NaN   
           Octavia  4,000.00       NaN   6,900.00 27,400.00  3,500.00   
Toyota     Camry    7,900.00  2,400.00   1,000.00       NaN       NaN   
           Corolla       NaN  8,594.59        NaN 24,000.00  4,400.00   
Volkswagen Golf     5,800.00       NaN  27,700.00 24,000.00  1,000.00   
           Jetta         NaN       NaN        NaN       NaN       NaN   
           Passat     100.00       NaN        NaN       NaN  2,000.00   
           Touareg       NaN    500.00        NaN       NaN       NaN   

Year                     1985       1986       1987       1988       1989  \
Make       Model                                                            
Ford       Focus   171,400.00 227,589.17 102,589.17 105,494.59 100,594.59   
           Mondeo         NaN  46,200.00  34,400.00        NaN        NaN   
Skoda      Focus          NaN        NaN        NaN        NaN        NaN   
           Octavia  10,600.00   5,800.00  75,900.00     300.00  14,200.00   
Toyota     Camry          NaN   7,200.00        NaN   2,200.00     800.00   
           Corolla        NaN        NaN        NaN  13,000.00  57,700.00   
Volkswagen Golf     13,194.59  30,900.00        NaN        NaN  10,600.00   
           Jetta          NaN        NaN   9,000.00        NaN        NaN   
           Passat      300.00  17,900.00        NaN        NaN        NaN   
           Touareg        NaN        NaN        NaN        NaN        NaN   

Year                ...      2010       2011       2012       2013       2014  \
Make       Model    ...                                                         
Ford       Focus    ... 75,200.00 121,289.17 218,294.59 292,483.76  71,883.76   
           Mondeo   ...       NaN        NaN        NaN        NaN        NaN   
Skoda      Focus    ...       NaN        NaN   1,600.00        NaN        NaN   
           Octavia  ...  8,594.59   3,000.00 156,094.59   3,900.00  15,594.59   
Toyota     Camry    ...  7,500.00        NaN  13,000.00  12,000.00        NaN   
           Corolla  ...       NaN        NaN  12,700.00   1,000.00        NaN   
Volkswagen Golf     ...       NaN     500.00        NaN        NaN 168,000.00   
           Jetta    ...       NaN        NaN        NaN  46,500.00   1,000.00   
           Passat   ...       NaN   8,594.59        NaN     500.00        NaN   
           Touareg  ...       NaN        NaN        NaN        NaN        NaN   

Year                     2015       2016       2017       2018       2019  
Make       Model                                                           
Ford       Focus   161,194.59 269,194.59 291,794.59 150,500.00 171,889.17  
           Mondeo         NaN        NaN   8,600.00        NaN        NaN  
Skoda      Focus          NaN        NaN        NaN        NaN        NaN  
           Octavia        NaN        NaN   3,000.00  10,800.00        NaN  
Toyota     Camry          NaN        NaN  19,800.00  10,600.00        NaN  
           Corolla        NaN  25,900.00   8,900.00        NaN   6,800.00  
Volkswagen Golf           NaN        NaN   2,300.00  29,800.00  12,000.00  
           Jetta          NaN        NaN        NaN        NaN        NaN  
           Passat   11,400.00   9,900.00        NaN        NaN        NaN  
           Touareg        NaN        NaN        NaN        NaN        NaN  

[10 rows x 40 columns]

In [285]:
fines.to_csv('..\\data\\fines.csv', index=False)
owners.to_csv('..\\data\\owners.csv', index=False)